# Dream Destination Finder with CrewAI and Amazon Bedrock

In this notebook, we will explore how to use the CrewAI framework with Amazon Bedrock to build an intelligent agent that can find dream travel destinations based on user preferences. The agent will utilize a large language model (LLM) and web search capabilities to research and recommend destinations that match the user's description.

### Prerequisites

Before we begin, make sure you have the following installed:
`boto3` and `botocore` for interacting with AWS services
`crewai` and `crewai_tools` for building agentic workflows

#### Configuring AWS Credentials:
Before using Amazon Bedrock, ensure that your AWS credentials are configured correctly. You can set them up using the AWS CLI or by setting environment variables. For this notebook, we’ll assume that the credentials are already configured.

To use bedrock we will use [__CrewAI__ __LLM__ api](https://docs.crewai.com/how-to/llm-connections#supported-providers) 

In [20]:
import boto3
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

#### Define web-search tool

In [21]:
@tool('DuckDuckGoSearch')
def search(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

### Configuring the LLM

We will use Amazon Nova Lite as our LLM. CrewAI uses LiteLLM under the hood to interact with different LLM providers.


In [44]:
# Configure the LLM
llm = LLM(model="us.amazon.nova-pro-v1:0")

### Defining the Agent

We will create an agent with the role of a “Travel Destination Researcher.” This agent will be responsible for finding destinations that match the user’s travel preferences.

In [45]:
# Define the Agent
travel_agent = Agent(
    role='Travel Destination Researcher',
    goal='Find dream destinations matching user preferences',
    backstory="You are an experienced travel agent specializing in personalized travel recommendations.",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[search]  # Tool for online searching
)

### Defining the Task

We need to specify the task that the agent will perform. The task includes a description, expected output, and is assigned to the agent we just created.

In [46]:
# Define the Task
task = Task(
    description="Based on the user's travel preferences: {preferences}, research and recommend suitable travel destinations.",
    expected_output="A list of recommended destinations with brief descriptions.",
    agent=travel_agent
)

### Creating the Crew

A crew is a team of agents working together to achieve a common goal. In this case, we have only one agent, but the framework allows for scalability.


In [47]:
# Create the Crew
crew = Crew(
    agents=[travel_agent],
    tasks=[task],
    verbose=True,
)

### Executing the Workflow

Now, we can execute the crew with the user’s travel preferences as input.

In [48]:
# User input for travel preferences
user_input = {
    "preferences": "I want a tropical beach vacation with great snorkeling and vibrant nightlife."
}

# Execute the Crew
result = crew.kickoff(inputs=user_input)

# Agent: Travel Destination Researcher
## Task: Based on the user's travel preferences: I want a tropical beach vacation with great snorkeling and vibrant nightlife., research and recommend suitable travel destinations.


# Agent: Travel Destination Researcher
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"top tropical beach destinations for snorkeling and nightlife\"}"
## Tool Output: 
Ilha Grande, a stunning Brazilian island, is celebrated for its lush tropical forests and pristine beaches. The island's waters are rich in marine life, featuring colorful fish, sea turtles, and starfish. Snorkeling spots like Lagoa Azul, or Blue Lagoon, provide crystal-clear waters and a wealth of biodiversity, making it an ideal destination. 9 Active Beach Vacations With Snorkeling, Hiking, and Endless Water Sports ... the globe for the best active beach vacations, from over-water rock climbing in Thailand to hiking in Acadia National ... Fiji is known as the soft coral capital 

#### As the crew executes, CrewAI will:

•	Decompose the task into actions using ReAct (Reasoning and Act), optionally using the tools assigned to the agent.

•	Make multiple calls to Amazon Bedrock to complete each step from the previous phase.

In [49]:
from IPython.display import Markdown

In [50]:
Markdown(result.raw)

Here are some recommended tropical beach destinations that offer great snorkeling and vibrant nightlife:

1. **Fiji**: Known for its soft coral capital status with over 390 coral species, Fiji offers snorkeling accessible almost anywhere throughout its 300 islands and 500 plus reefs and inlets.

2. **Phuket, Thailand**: Offers serene beaches like Kata Noi and lively nightlife at Patong, making it an unbeatable tropical escape. It's also known for its Thai cuisine and affordable luxury.

3. **Puerto Rico**: Combines nightlife, beaches, and world-class cuisine, making it one of the best tropical vacation destinations.

4. **Great Barrier Reef, Australia**: From majestic rays to sea turtles and colorful tropical fish, it's one of the world's best snorkeling sites.

5. **Tofo Beach, Mozambique**: Best for snorkeling with whale sharks and offers a variety of water activities including tennis, beach volleyball, sailing, windsurfing, kayaking, and snorkeling.

6. **Ilha Grande, Brazil**: Famous for its lush tropical forests, pristine beaches, and vibrant marine life, including colorful fish, sea turtles, and starfish. Snorkeling spots like Lagoa Azul, or Blue Lagoon, provide crystal-clear waters and a wealth of biodiversity.

These destinations should provide a perfect blend of tropical beaches, excellent snorkeling opportunities, and vibrant nightlife.

### Adding Memory to the Agent
CrewAI supports [several memory types](https://docs.crewai.com/concepts/memory#implementing-memory-in-your-crew), which help agents remember and learn from past interactions. In this case, we’ll enable short-term memory using Amazon Bedrock’s embedding model.

In [51]:
boto3_session = boto3.Session(
    region_name="us-west-2"
    )
bedrock_runtime = boto3_session.client(service_name="bedrock-runtime")

embedder={
       "provider": "bedrock",
        "config": {
            "session": boto3_session,
            "model": "amazon.titan-embed-text-v2:0",
            "dimension": 1024
        }
    }

# Enabling Memory in the Agent
crew_with_memory = Crew(
    agents=[travel_agent],
    tasks=[task],
    verbose=True,
    memory=True,  # Enable memory
    embedder=embedder
    
)

In [34]:
# Executing the Crew with Memory
result_with_memory = crew_with_memory.kickoff(inputs=user_input)

# Agent: Travel Destination Researcher
## Task: Based on the user's travel preferences: I want a tropical beach vacation with great snorkeling and vibrant nightlife., research and recommend suitable travel destinations.


# Agent: Travel Destination Researcher
## Using tool: DuckDuckGoSearch
## Tool Input: 
"{\"search_query\": \"tropical beach vacation with great snorkeling and vibrant nightlife destinations\"}"
## Tool Output: 
The couples resort Sandals Grenada has rooms with private infinity pools fronting Pink Gin Beach that set the scene pretty perfectly for a tropical beach vacation to remember. 18 of 30 Dominica Final Thoughts on the Best Tropical Island Vacations [Previous content remains the same] Conclusion: Your Paradise Awaits. These 20 stunning tropical islands represent the pinnacle of beach vacation destinations, offering unparalleled beauty, diverse experiences, and unforgettable memories. 9 Active Beach Vacations With Snorkeling, Hiking, and Endless Water Sports ... of

In [35]:
Markdown(result_with_memory.raw)

Here are some recommended destinations for a tropical beach vacation with great snorkeling and vibrant nightlife:

1. **Maldives** - Known for its crystal-clear waters and vibrant coral reefs, the Maldives offers excellent snorkeling opportunities. The capital, Malé, has a lively nightlife scene with bars, clubs, and restaurants.

2. **Bora Bora, French Polynesia** - This tropical paradise is famous for its stunning beaches, overwater bungalows, and vibrant marine life. The island offers a range of snorkeling excursions and a lively nightlife scene in the main town of Vaitape.

3. **Phuket, Thailand** - Phuket is a popular destination for beach lovers and offers great snorkeling opportunities in the Andaman Sea. The island has a vibrant nightlife scene with bars, clubs, and live music venues.

4. **Aruba, Dutch Caribbean** - Aruba is known for its pristine beaches, crystal-clear waters, and vibrant nightlife. The island offers excellent snorkeling opportunities and a range of beach bars and clubs.

5. **Cayman Islands** - The Cayman Islands are known for their crystal-clear waters and vibrant coral reefs. The islands offer excellent snorkeling opportunities and a lively nightlife scene in the capital, George Town.

6. **Grenada** - Known for its beautiful beaches and crystal-clear waters, Grenada offers excellent snorkeling opportunities. The capital, St. George's, has a vibrant nightlife scene with bars, clubs, and live music venues.

7. **Dominica** - This Caribbean island is known for its natural beauty and vibrant marine life. The island offers excellent snorkeling opportunities and a range of eco-tours and adventure activities.

8. **Fiji** - Fiji is a tropical paradise known for its stunning beaches, crystal-clear waters, and vibrant coral reefs. The island offers a range of snorkeling excursions and a lively nightlife scene in the capital, Suva.

9. **Costa Rica** - The Pacific coast of Costa Rica offers beautiful beaches and vibrant marine life. The country offers excellent snorkeling opportunities and a range of eco-tours and adventure activities.

10. **Belize** - Known for its vibrant marine life and beautiful coral reefs, Belize offers excellent snorkeling opportunities. The country has a range of beach resorts and a lively nightlife scene in the capital, Belize City.

### Integrating Retrieval-Augmented Generation (RAG) with Amazon Bedrock Knowledge Base
In this section, we will enhance our dream destination finder agent by incorporating Retrieval-Augmented Generation (RAG) using Amazon Bedrock’s Knowledge Base. This will allow our agent to access up-to-date and domain-specific travel information, improving the accuracy and relevance of its recommendations.



#### What is Retrieval-Augmented Generation (RAG)?

RAG is a technique that combines the capabilities of large language models (LLMs) with a retrieval mechanism to fetch relevant information from external data sources. By integrating RAG, our agent can retrieve the most recent and specific information from a knowledge base, overcoming the limitations of LLMs that may have outdated or insufficient data.

Setting Up Amazon Bedrock Knowledge Base

Before we proceed, ensure you have access to Amazon Bedrock and the necessary permissions to create and manage knowledge bases.

* Step 1: Prepare Your Data
* Step 2: Create a Knowledge Base in Amazon Bedrock
* Step 3: Note the Knowledge Base ID

After the knowledge base is created, note down its Knowledge Base ID (kb_id), which will be used in our code.

Updating the Agent to Use RAG with CrewAI

We will modify our agent to include a custom tool that queries the Amazon Bedrock Knowledge Base. This allows the agent to retrieve up-to-date information during its reasoning process.

### FAIS Vector Store Set up:

In [38]:
import os
from uuid import uuid4
from PyPDF2 import PdfReader
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain_aws import BedrockEmbeddings

documents = []
pdf_folder = 'data/travel_guides'

# Loop through PDFs in the specified folder
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.endswith(".pdf"):
        file_path = os.path.join(pdf_folder, pdf_file)
        
        # Extract text from PDF
        reader = PdfReader(file_path)
        text_content = ""
        for page in reader.pages:
            text_content += page.extract_text() + "\n"
        
        # Create a Document instance
        doc = Document(
            page_content=text_content.strip(),
            metadata={}  # Leave metadata empty for now
        )
        documents.append(doc)

# Initialize FAISS vector store and embeddings
embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0")
vector_store = FAISS.from_documents(documents, embeddings)

# Add unique IDs to documents and save the vector store
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['9832925e-4a14-4847-8c73-97b41e70be45',
 'c41c15a8-d729-44e9-a763-1f5761ee5b01',
 'a08d0572-11c9-4b15-92cb-e10eb94abbac',
 '2898513e-49ec-4728-b571-b8b74f14a8bc',
 '13b30ea7-bce2-4c5b-8fd4-b848ff82819c',
 'bcba276a-27cf-457e-a52c-8c7921373be8',
 '6bc12bf1-c1fd-41e0-b04f-13a8fb47b9cf',
 'e56613f9-e712-48ba-891d-7b4f83eecda7',
 '32b162a7-2981-42f4-b83c-a16cb4a633c1',
 '48959d9f-9d55-40ad-919a-f0ea95d00c45',
 '72b54ec9-754e-4031-8f95-b23ac06205ad',
 '18847ff5-a905-42d7-9854-f19405f91e95',
 '64dc1cc3-e057-499f-9c88-dd73ed41a15f']

In [39]:
@tool("TravelExpertSearchEngine")
def query_knowledge_base(question: str) -> str:
    """Queries the Amazon Bedrock Knowledge Base for travel-related information."""
    try:
        res = vector_store.similarity_search(
        question,
        k=1,
        )        
        return res[0].page_content
    except KeyError:
        return "No data available"



### Update the Agent with the New Tool
We will update our agent to include the TravelExpert tool.

In [40]:
# Update the Agent
agent_with_rag = Agent(
    role='Travel Destination Researcher',
    goal='Find dream destinations in the USA using only the travel guide available, first lookup cities using the tool to match user preferences and then use information from the search engine, nothing else.',
    backstory="""You are an experienced travel agent specializing in personalized travel recommendations. 
                 Your approach is as follows: 
                 Deduce which regions within the USA will have those activities listed by the user.
                 List major cities within that region
                 Only then use the tool provided to look up information, look up should be done by passing city highlights and activities.
                 Only suggest places that were extracted using the lookup tool,
              """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    tools=[query_knowledge_base],  # Include the RAG tool
    max_iter=3
)


### Update the task and set up the Crew

In [41]:
# Define the Task
task_with_rag = Task(
    description="Based on the user's travel request, research and recommend suitable travel destinations using the latest information. Only use output provided by the Travel Destination Researcher, nothing else: USER: {preferences}",
    expected_output="A place where they can travel to along with recommendations on what to see and do while there.",
    agent=agent_with_rag
)


# Create the Crew
crew_with_rag = Crew(
    agents=[agent_with_rag],
    tasks=[task_with_rag],
    verbose=True,
)

In [42]:
# User input for travel preferences
user_input = {
    "preferences": "Where can I go for cowboy vibes, watch a rodeo, and a visit museums?"
}

# Execute the Crew
result_with_rag = crew_with_rag.kickoff(inputs=user_input)


# Agent: Travel Destination Researcher
## Task: Based on the user's travel request, research and recommend suitable travel destinations using the latest information. Only use output provided by the Travel Destination Researcher, nothing else: USER: Where can I go for cowboy vibes, watch a rodeo, and a visit museums?


# Agent: Travel Destination Researcher
## Using tool: [TravelExpertSearchEngine]
## Tool Input: 
"{\"question\": \"What are the major cities in the Western United States known for cowboy culture and rodeos?\"}"
## Tool Output: 
Travel Guide: Dallas
Generated by Llama3.1 405B
 
Dallas, the vibrant heart of Texas, is a city that captivates with its dynamic blend of modern
sophistication and cowboy charm. As the ninth-largest city in the United States, Dallas dazzles visitors
with its towering skyscrapers, world-class museums, and thriving arts scene.
 
Explore the iconic Reunion Tower, where the observation deck offers panoramic views of the city
skyline. Immerse yourself i

### Display the results

In [43]:
# Display the result
Markdown(result_with_rag.raw)

Based on the user's request for cowboy vibes, watching a rodeo, and visiting museums, the best recommendation is to visit Fort Worth, Texas. Fort Worth is known for its vibrant cowboy culture and hosts the famous Fort Worth Stock Show and Rodeo. Additionally, Fort Worth is home to several museums, including the Amon Carter Museum of American Art and the Fort Worth Museum of Science and History. These attractions provide a perfect blend of cowboy culture, rodeo events, and museum visits.
```